# import packages

In [1]:
import numpy as np
import pandas as pd
import string
import random
from langdetect import detect
import re

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# initial exploration/summary

In [4]:
train = pd.read_csv(r'C:\Users\caseyrya\Dropbox\foursquare_location_matching_data\data\train.csv')

train['lat_long'] = [', '.join(str(x) for x in y) for y in map(tuple, train[['latitude', 'longitude']].values)]

print(train.shape)
train.head()

(1138812, 14)


id                     name   latitude   longitude  \
0  E_000001272c6c5d     Café Stad Oudenaarde  50.859975    3.634196   
1  E_000002eae2a589           Carioca Manero -22.907225  -43.178244   
2  E_000007f24ebc95         ร้านตัดผมการาเกด  13.780813  100.484900   
3  E_000008a8ba4f48                 Turkcell  37.844510   27.844202   
4  E_00001d92066153  Restaurante Casa Cofiño  43.338196   -4.326821   

                  address        city            state   zip country  url  \
0             Abdijstraat  Nederename  Oost-Vlaanderen  9700      BE  NaN   
1                     NaN         NaN              NaN   NaN      BR  NaN   
2                     NaN         NaN              NaN   NaN      TH  NaN   
3  Adnan Menderes Bulvarı         NaN              NaN   NaN      TR  NaN   
4                     NaN    Caviedes        Cantabria   NaN      ES  NaN   

  phone             categories point_of_interest  \
0   NaN                   Bars  P_677e840bb6fc7e   
1   NaN  Brazilian Restaurants  P_d82910d8382a83   
2   NaN   Salons / Barbershops  P_b1066599e78477   
3   NaN     Mobile Phone Shops  P_b2ed86905a4cd3   
4   NaN    Spanish Restaurants  P_809a884d4407fb   

                                  lat_long  
0    50.85997533509319, 3.6341963487748346  
1  -22.907225306597013, -43.17824390023254  
2   13.780813251617689, 100.48489995223518  
3     37.84451021530735, 27.84420157619721  
4                     43.338196, -4.326821

In [ ]:
print(f'duplicate rows: {len(train[train.duplicated()])}')
print()
print()
for col in train.columns:
    print(f'column data type "{col}": {train[col].dtype}')
    print(f'missing values in "{col}": {len(train[train[col].isna()])}')
    print(f'unique values in "{col}": {len(train[train[col].isna() == False][col].unique())}')
    print(f'non-duplicated values in "{col}": {len(train[(train[col].isna() == False) & (train[col].duplicated(keep=False) == False)])}')
    print(f'duplicated values in "{col}": {len(train[(train[col].isna() == False) & (train[col].duplicated())][col].drop_duplicates())}')
    print(f'duplicated values row count in "{col}": {len(train[(train[col].isna() == False) & (train[col].duplicated(keep = False))])}')
    print()

# exploration

In [5]:
eda = train.copy()

eda.head()

id                     name   latitude   longitude  \
0  E_000001272c6c5d     Café Stad Oudenaarde  50.859975    3.634196   
1  E_000002eae2a589           Carioca Manero -22.907225  -43.178244   
2  E_000007f24ebc95         ร้านตัดผมการาเกด  13.780813  100.484900   
3  E_000008a8ba4f48                 Turkcell  37.844510   27.844202   
4  E_00001d92066153  Restaurante Casa Cofiño  43.338196   -4.326821   

                  address        city            state   zip country  url  \
0             Abdijstraat  Nederename  Oost-Vlaanderen  9700      BE  NaN   
1                     NaN         NaN              NaN   NaN      BR  NaN   
2                     NaN         NaN              NaN   NaN      TH  NaN   
3  Adnan Menderes Bulvarı         NaN              NaN   NaN      TR  NaN   
4                     NaN    Caviedes        Cantabria   NaN      ES  NaN   

  phone             categories point_of_interest  \
0   NaN                   Bars  P_677e840bb6fc7e   
1   NaN  Brazilian Restaurants  P_d82910d8382a83   
2   NaN   Salons / Barbershops  P_b1066599e78477   
3   NaN     Mobile Phone Shops  P_b2ed86905a4cd3   
4   NaN    Spanish Restaurants  P_809a884d4407fb   

                                  lat_long  
0    50.85997533509319, 3.6341963487748346  
1  -22.907225306597013, -43.17824390023254  
2   13.780813251617689, 100.48489995223518  
3     37.84451021530735, 27.84420157619721  
4                     43.338196, -4.326821

In [78]:
# for comparing old vs. new fields

def clean_text_field_both(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    # df[new_text_field_name] = df[new_text_field_name].map(lambda x: re.sub(r'[^\w\s]', '', x) if type(x) == object else x)
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
    # df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    
    return df

In [74]:
# for production

def clean_text_field(df, field):
    df[field] = df[field].str.lower() # map to lower case
    df[field] = df[field].apply(lambda x: re.sub(r'[^\w\s]', '', str(x))) # remove anything that isn’t a word or a whitespace

    # df[field] = df[field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    
    return df

In [98]:
# df = eda[eda['address'].isnull()==False][['id','address']].copy()
col = 'zip'

df = eda[['id',col]].copy()
print(df.shape)

df = clean_text_field(df, col)
df[df[col].isnull()==False].head(10)

(1138812, 2)


id    zip
0  E_000001272c6c5d   9700
1  E_000002eae2a589    nan
2  E_000007f24ebc95    nan
3  E_000008a8ba4f48    nan
4  E_00001d92066153    nan
5  E_000023d8f4be44    nan
6  E_00002a131a2bf6    nan
7  E_0000764d65557e    nan
8  E_00007dcd2bb53f  94534
9  E_0000890af22ff5    nan

In [96]:
df_ll = eda[['id','latitude','longitude']].copy()

df_ll['latitude'] = round(df_ll['latitude'], 6)
df_ll['longitude'] = round(df_ll['longitude'], 6)

# df['col'].astype(str)

df_ll.head()

id   latitude   longitude
0  E_000001272c6c5d  50.859975    3.634196
1  E_000002eae2a589 -22.907225  -43.178244
2  E_000007f24ebc95  13.780813  100.484900
3  E_000008a8ba4f48  37.844510   27.844202
4  E_00001d92066153  43.338196   -4.326821

In [97]:
df_ll['latitude'][0]

50.859975

In [92]:
df_ll.dtypes

id            object
latitude     float64
longitude    float64
dtype: object

In [99]:
test = eda[eda['phone'].isnull()==False][['id', 'phone']].copy()

test['phone_len'] = test['phone'].str.len()

test.sort_values('phone_len', ascending = False).head(100)

id                                        phone  \
433297   E_6165f823420990  1014 (Particuliers) - 1016 (Professionnels)   
732108   E_a473b63104d9b6  1014 (Particuliers) - 1016 (Professionnels)   
356977   E_503d20f0a8a8bf  1014 (Particuliers) - 1016 (Professionnels)   
83334    E_12a892f5a08db8  1014 (Particuliers) - 1016 (Professionnels)   
924443   E_cfa8b5197df43d  1014 (Particuliers) - 1016 (Professionnels)   
33792    E_0790cf7e60b85f    (383) 2492000 (383) 2492101 (383) 2492303   
355851   E_4ffb333e337799      924-82-24 925-29-41 925-35-99 925-28-63   
356183   E_500cc2bb71f1bc      773-12-82 773-13-16 773-13-18 773-13-19   
471649   E_69ffe3a1ef6ca8      232-16-12 232-16-18 232-16-41 232-56-90   
1111546  E_f9dc1f03c48c49          57-52-80-61 57-52-85-27 57-52-73-39   
552117   E_7c182998031b21           6336288496863362884765639228997651   
415928   E_5d7165452bc3e7           6372242454463722426149639178803040   
869158   E_c33f19c3fb89a0           6336288310163362883112639216653672   
10256    E_025583f1c5a4d9           6338502212263385022122639173043057   
9495     E_02280541425fb7              (383) 249-40-30 (383) 249-40-60   
595714   E_85e67483cbbd08              8 (7182) 651214 8 (7182) 530521   
130477   E_1d41c287ce01cb              2474-8184, 2474-8185, 2474-8193   
1004559  E_e1a86aed110f2e               +852 2522 5343/ +852 2552 5627   
366157   E_52427e9ff4c86d                01(66)22515293/01(66)22515294   
763988   E_ab9d6457a72052                  (0155) 5562 1921/5562 22 50   
1047322  E_eb5487858acaa2                   (495) 258 54 57, 258 54 58   
780024   E_af32857c36a78d                    (0322) 45-28-24, 45-28-25   
510655   E_72ca6b22a76255                    083-688-7695, 02-730-8332   
1057266  E_ed978e4d822dd8                    (8555) 24-59-02, 24-59-03   
1131799  E_fe6ab6d7e1e86c                     (0324) 55-80-70, 5-08-64   
905715   E_cb7432e958483f                     01 (78) 2111 5636 / 5640   
672792   E_972352440507c3                     077-282115 , 089 7293808   
1053386  E_ecb176518e95f6                     051262526100051262527100   
951314   E_d5b355c3b342a5                     051265322388051265575100   
990987   E_de9db8477b7276                     +49 (0) 64 03 / 96 93 40   
630876   E_8dda7471d18261                      63344763019639173609087   
47693    E_0aab61fb43a8b6                      63842173792639202981318   
1066610  E_efb7f73ee4d966                      01(55) 5260 3861 / 3830   
653491   E_92d9f7f9c8b50a                      63888571900639058898229   
13283    E_0300cc0b8020d4                      01(46) 4647 1401 / 6383   
294935   E_423c339ad1948b                       +86 (512) 88855288-888   
185916   E_29b662c60dea95                       6332341458663323414595   
171322   E_2668b4f3055f19                       6332232311163322323117   
147624   E_2119b7d216d2b7                       8802883600088028836444   
137710   E_1ed8dadb54afbf                       6332232311163322323117   
747705   E_a7fac52e178df3                       6332255601063322532893   
578932   E_82234d05e2bd42                       6336288408563362884085   
934610   E_d1f091202402f2                        026176486,87,88,89,90   
1018003  E_e4b6332af151ef                        281108628110852811080   
1039315  E_e981e051b12f4d                        634640310006328864325   
180209   E_286eeaab1eb585                        011886223315256511512   
443551   E_63af451d239a7b                        022723103102276513854   
836123   E_bbc0fb8ef267c7                        (303) 471-WING (9464)   
149326   E_217ad0e56d8023                        010626667748009880153   
873205   E_c421ea5ac562d8                         01(81) 82409110 / 09   
697593   E_9caf76c2d98991                         01 66 22 64 59 61/62   
532041   E_7792baa336462f                         (044) 542-71-18 (17)   
700717   E_9d5fee0d7be819                         227-74717, 2278-6725   
730547   E_a41a7da4691fef  

# dev stuff

In [16]:
from langdetect import detect
from langdetect import detect_langs

# def detect_lang(text):
#     return detect(text)

# name_eda['name'].apply(detect_lang)

print(detect('ร้านตัดผมการาเกด'))
print(detect('ビックカメラ JR京都駅店'))
print(detect('Hello Max'))
print(detect('i am the automation'))
print(detect("Otec matka syn."))
print()


print(detect_langs('ร้านตัดผมการาเกด'))
print(detect_langs('ビックカメラ JR京都駅店'))
print(detect_langs('Hello Max'))
print(detect_langs('i am the automation'))
print(detect_langs("Otec matka syn."))

th
ja
it
en
fi

[th:0.9999999944591088]
[ja:0.9999998795307918]
[it:0.7142847028502886, so:0.28571483004638965]
[en:0.9999968395242242]
[fi:0.5714255656124794, pl:0.42857140739166466]
